In [1]:
!pip install pandas numpy

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import numpy as np


In [9]:
df = pd.read_csv("C:\\sentrymind\\backend\\data\\fraudTrain.csv")


In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [13]:
print(df.head())

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ...      lat      long  \
0    Banks      F                561 Perry Cove  ...  36.0788  -81.1781   
1     Gill      F  43039 Riley Greens Suite 393  ...  48

In [15]:
# Handle Missing & Duplicate Values
df = df.dropna()  # Drop rows with missing values
df = df.drop_duplicates(subset=['trans_num'])  # Ensure unique transaction IDs

In [17]:
# Convert Data Types
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['is_fraud'] = df['is_fraud'].astype(bool)
df['amt'] = df['amt'].astype(float)

In [19]:
# Standardize Merchant & Location Data
df['merchant'] = df['merchant'].str.lower()
df[['lat', 'long', 'merch_lat', 'merch_long']] = df[['lat', 'long', 'merch_lat', 'merch_long']].astype(float)

In [21]:
# Derive Additional Features
df['year'] = df['trans_date_trans_time'].dt.year
df['month'] = df['trans_date_trans_time'].dt.month
df['day'] = df['trans_date_trans_time'].dt.day
df['hour'] = df['trans_date_trans_time'].dt.hour

In [25]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

In [27]:
# Sort by customer ID and transaction timestamp
df = df.sort_values(by=['cc_num', 'trans_date_trans_time'])

In [31]:
# Function to compute transactions in last 24 hours
def count_transactions_last_24h(group):
    return group.rolling('1D', on='trans_date_trans_time').count()


In [33]:
df['transaction_count_24h'] = df.groupby('cc_num', group_keys=False).apply(count_transactions_last_24h)['trans_num']

In [37]:
# Generate a synthetic merchant risk score
df['merchant_risk_score'] = np.random.uniform(1, 10, df.shape[0])

In [39]:
df.to_csv("C:\\sentrymind\\backend\\data\\fraudDetection_cleaned.csv", index=False)

In [42]:
from sqlalchemy import create_engine

In [44]:
MYSQL_USER = "root"
MYSQL_PASSWORD = "root"
MYSQL_HOST = "localhost"  # Change if using a remote server
MYSQL_DATABASE = "sentrymind"

In [48]:
!pip install pymysql

   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 217.9 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 318.6 kB/s eta 0:00:00


In [50]:
engine = create_engine(f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DATABASE}")

In [52]:
df = pd.read_csv("C:\\sentrymind\\backend\\data\\fraudDetection_cleaned.csv")

In [54]:
df.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,unix_time,merch_lat,merch_long,is_fraud,year,month,day,hour,transaction_count_24h,merchant_risk_score
0,1017,2019-01-01 12:47:15,60416207185,"fraud_jones, sawayn and romaguera",misc_net,7.27,Mary,Diaz,F,9886 Anita Drive,...,1325422035,43.974711,-109.741904,False,2019,1,1,12,1.0,6.973019
1,2724,2019-01-02 08:44:57,60416207185,fraud_berge llc,gas_transport,52.94,Mary,Diaz,F,9886 Anita Drive,...,1325493897,42.018766,-109.044172,False,2019,1,2,8,2.0,1.123834
2,2726,2019-01-02 08:47:36,60416207185,fraud_luettgen plc,gas_transport,82.08,Mary,Diaz,F,9886 Anita Drive,...,1325494056,42.961335,-109.157564,False,2019,1,2,8,3.0,5.508184
3,2882,2019-01-02 12:38:14,60416207185,fraud_daugherty llc,kids_pets,34.79,Mary,Diaz,F,9886 Anita Drive,...,1325507894,42.228227,-108.747683,False,2019,1,2,12,4.0,6.405251
4,2907,2019-01-02 13:10:46,60416207185,fraud_beier and sons,home,27.18,Mary,Diaz,F,9886 Anita Drive,...,1325509846,43.321745,-108.091143,False,2019,1,2,13,4.0,3.175471


In [56]:
print("CSV Headers:", list(df.columns))


CSV Headers: ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'year', 'month', 'day', 'hour', 'transaction_count_24h', 'merchant_risk_score']


In [60]:
df.drop(columns=['Unnamed: 0', 'year', 'month', 'day', 'hour'], inplace=True)

In [62]:
# Rename columns to match MySQL table
df.rename(columns={
    "trans_date_trans_time": "trans_date",
    "amt": "amount",
    "first": "first_name",
    "last": "last_name",
    "zip": "zip_code",
    "lat": "latitude",
    "long": "longitude",
    "city_pop": "city_population",
    "merch_lat": "merch_latitude",
    "merch_long": "merch_longitude"
}, inplace=True)

In [64]:
df.to_csv("C:\\sentrymind\\backend\\data\\fraudDetection_cleaned.csv", index=False)


In [66]:
df = pd.read_csv("C:\\sentrymind\\backend\\data\\fraudDetection_cleaned.csv")

In [68]:
df.to_sql(name="transactions", con=engine, if_exists="append", index=False)

1296675